In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup

# 1. Scrap Site www.topuniversities.com

found the json using postman:

https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508252855868

We want:
    - name 
    - rank
    - country
    - region 
    - number of faculty members (international and total)
    - number of students (international and total)


In [2]:
#json to dataframe using panda
json_data = json.load(open('TopUniRanking.json'))
df = json_normalize(json_data['data'])
#the university are sorted by ranks, so we just need the 200 first indexes
df_top200 = df.head(200)
#drop useless column information
df_top200 = df_top200.drop(["cc","core_id","logo","guide"],axis=1)
df_top200.head(10)

,country,nid,rank_display,region,score,stars,title,url
0,United States,294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...
1,United States,297282,2,North America,98.7,5,Stanford University,/universities/stanford-university
2,United States,294270,3,North America,98.4,5,Harvard University,/universities/harvard-university
3,United States,294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...
4,United Kingdom,294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge
5,United Kingdom,294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford
6,United Kingdom,294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london
7,United Kingdom,294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london
8,United States,294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago
9,Switzerland,294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...


We now have the general data for the 200 first universities. Then we need to scrap data on each unique page of each university to retrieve those 4 informations: 
* number of total faculty members 
* number of international faculty members
* number of total students 
* number of international students

The url of an university is contained in our previous dataFrame. And the pages are written this way:
 **www.topuniversities.com+url** 



In [3]:
listUrl = df_top200['url']
#create a temp dataframe
index = range(200)
columns = ["nbr_faculty_members","nbr_international_faculty_members","nbr_total_students","nbr_international_total_students"]
temp_df = pd.DataFrame(index=index,columns=columns)

#helper function that filter the html file and return the int corresponding to the special string htmlClass
def findNumberOf(soup,htmlClass):
    filteredHtmlElements = soup.find_all('div', class_=htmlClass)
    #case if a field information is not given
    if(len(filteredHtmlElements)==0):
        return np.nan
    #find the integer value inside the html balise
    t= filteredHtmlElements[0].find('div', class_='number').text
    #clear the input then convert it into an integer
    return int(t.replace('\n', '').replace('\r', '').replace(',', '').replace(' ',''))

#retrieve the 4 informations needed from the url, for each university
for i,url in enumerate(listUrl):
    finalUrl = "https://www.topuniversities.com"+url
    r = requests.get(finalUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    #todo: scrap with beautifulSoup using the url
    #merge data into the temp dataframe
    temp_df.nbr_faculty_members[i] = findNumberOf(soup,'total faculty')
    temp_df.nbr_international_faculty_members[i] = findNumberOf(soup,'inter faculty')
    temp_df.nbr_total_students[i] = findNumberOf(soup,'total student')
    temp_df.nbr_international_total_students[i] = findNumberOf(soup,'total inter')

In [4]:
#merge the 2 temps
df_top200_merged = df_top200.join(temp_df)
df_top200_merged.head(10)

,country,nid,rank_display,region,score,stars,title,url,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students
0,United States,294850,1,North America,100,6,Massachusetts Institute of Technology (MIT),/universities/massachusetts-institute-technolo...,2982,1679,11067,3717
1,United States,297282,2,North America,98.7,5,Stanford University,/universities/stanford-university,4285,2042,15878,3611
2,United States,294270,3,North America,98.4,5,Harvard University,/universities/harvard-university,4350,1311,22429,5266
3,United States,294562,4,North America,97.7,5,California Institute of Technology (Caltech),/universities/california-institute-technology-...,953,350,2255,647
4,United Kingdom,294561,5,Europe,95.6,5,University of Cambridge,/universities/university-cambridge,5490,2278,18770,6699
5,United Kingdom,294654,6,Europe,95.3,5,University of Oxford,/universities/university-oxford,6750,2964,19720,7353
6,United Kingdom,294014,7,Europe,94.6,None,UCL (University College London),/universities/ucl-university-college-london,6345,2554,31080,14854
7,United Kingdom,294030,8,Europe,93.7,None,Imperial College London,/universities/imperial-college-london,3930,2071,16090,8746
8,United States,294536,9,North America,93.5,5,University of Chicago,/universities/university-chicago,2449,635,13557,3379
9,Switzerland,294432,10,Europe,93.3,None,ETH Zurich - Swiss Federal Institute of Techno...,/universities/eth-zurich-swiss-federal-institu...,2477,1886,19815,7563


* Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
* Answer the previous question aggregating the data by (c) country and (d) region.

In [5]:
#drop university that doesn't have the infromation required
yolo_df = df_top200_merged.copy().drop(["nid","stars","url"],axis=1).dropna()
#df_top200_merged
yolo_df
def percentFacAndStud (row):
    return row['nbr_faculty_members']/row['nbr_total_students']
yolo_df['percent_fac_stud'] = yolo_df.apply (lambda row: percentFacAndStud (row),axis=1)
def percentInterAndStud (row):
    return row['nbr_international_total_students']/row['nbr_total_students']
yolo_df['percent_inter_stud'] = yolo_df.apply (lambda row: percentInterAndStud (row),axis=1)
yolo_df.sort_values(by='percent_fac_stud',ascending=False).head(10)

,country,rank_display,region,score,title,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students,percent_fac_stud,percent_inter_stud
3,United States,4,North America,97.7,California Institute of Technology (Caltech),953,350,2255,647,0.422616,0.286918
15,United States,16,North America,90.4,Yale University,4940,1708,12402,2469,0.398323,0.199081
5,United Kingdom,6,Europe,95.3,University of Oxford,6750,2964,19720,7353,0.342292,0.372870
4,United Kingdom,5,Europe,95.6,University of Cambridge,5490,2278,18770,6699,0.292488,0.356899
16,United States,17,North America,89.8,Johns Hopkins University,4462,1061,16146,4105,0.276353,0.254243
1,United States,2,North America,98.7,Stanford University,4285,2042,15878,3611,0.269870,0.227422
0,United States,1,North America,100,Massachusetts Institute of Technology (MIT),2982,1679,11067,3717,0.269450,0.335863
185,United States,186,North America,49.3,University of Rochester,2569,488,9636,2805,0.266604,0.291096
18,United States,19,North America,88.7,University of Pennsylvania,5499,1383,20639,4250,0.266437,0.205921
17,United States,18,North America,88.9,Columbia University,6189,913,25045,8105,0.247115,0.323617


In [6]:
yolo_df.sort_values(by='percent_inter_stud',ascending=False).head(10)

,country,rank_display,region,score,title,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students,percent_fac_stud,percent_inter_stud
34,United Kingdom,35,Europe,81.8,London School of Economics and Political Scien...,1088,687,9760,6748,0.111475,0.691393
11,Switzerland,12,Europe,91.2,Ecole Polytechnique Fédérale de Lausanne (EPFL),1695,1300,10343,5896,0.163879,0.570047
7,United Kingdom,8,Europe,93.7,Imperial College London,3930,2071,16090,8746,0.244251,0.543567
198,Netherlands,200,Europe,47.9,Maastricht University,1277,502,16385,8234,0.077937,0.502533
47,United States,=47,North America,78.6,Carnegie Mellon University,1342,425,13356,6385,0.100479,0.478062
6,United Kingdom,7,Europe,94.6,UCL (University College London),6345,2554,31080,14854,0.204151,0.477928
91,United Kingdom,92,Europe,65.3,University of St Andrews,1140,485,8800,4030,0.129545,0.457955
41,Australia,=41,Oceania,80.4,The University of Melbourne,3311,1477,42182,18030,0.078493,0.427434
126,United Kingdom,127,Europe,58.3,Queen Mary University of London,1885,801,16135,6806,0.116827,0.421816
25,Hong Kong,26,Asia,85.5,The University of Hong Kong,3012,2085,20214,8230,0.149006,0.407144


In [7]:
a = df_top200_merged.copy()[['country','region','nbr_faculty_members','nbr_international_faculty_members','nbr_total_students','nbr_international_total_students']]
b = a.drop('region',axis=1).groupby('country').agg('sum')
b['percent_fac_stud'] = b.apply (lambda row: percentFacAndStud (row),axis=1)
b['percent_inter_stud'] = b.apply (lambda row: percentInterAndStud (row),axis=1)
b.sort_values(by='percent_fac_stud',ascending=False).head(10)

,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students,percent_fac_stud,percent_inter_stud
country,,,,,,
Russia,6709,373,30233,5098,0.221910,0.168624
Denmark,11916,3904,67223,9543,0.177261,0.141960
Saudi Arabia,1062,665,6040,989,0.175828,0.163742
Singapore,9444,6079,58466,16168,0.161530,0.276537
Malaysia,2755,655,17902,3476,0.153893,0.194168
Japan,28395,2221,186222,16269,0.152479,0.087363
South Korea,19851,2010,140071,16273,0.141721,0.116177
Switzerland,15323,9208,109112,32995,0.140434,0.302396
United Kingdom,79934,30216,583621,199426,0.136962,0.341705


In [8]:
b.sort_values(by='percent_inter_stud',ascending=False).head(10)

,nbr_faculty_members,nbr_international_faculty_members,nbr_total_students,nbr_international_total_students,percent_fac_stud,percent_inter_stud
country,,,,,,
Australia,22034,11382,301994,106359,0.072962,0.352189
United Kingdom,79934,30216,583621,199426,0.136962,0.341705
Hong Kong,10166,6296,78838,24499,0.128948,0.310751
Austria,4117,1572,63446,19667,0.064890,0.309980
Switzerland,15323,9208,109112,32995,0.140434,0.302396
Singapore,9444,6079,58466,16168,0.161530,0.276537
Canada,29317,10734,281514,73239,0.104140,0.260161
New Zealand,3313,1404,48173,12439,0.068773,0.258215
Ireland,2853,1171,34794,8187,0.081997,0.235299


# 2. Scrap Site www.timeshighereducation.com

found the json using postman again:

https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json

We want:
    - name 
    - rank
    - country
    - region 
    - number of faculty members (international and total)
    - number of students (international and total)

In [9]:
#json to dataframe using panda
json_data2 = json.load(open('TimesRanking.json'))
df2 = json_normalize(json_data2['data'])
print(df2.T.index)
df2.head(10)

Index(['aliases', 'location', 'member_level', 'name', 'nid', 'rank',
       'rank_order', 'record_type', 'scores_citations',
       'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio', 'subjects_offered', 'url'],
      dtype='object')


,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,...,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,...,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,...,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,...,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,0,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,master_account,94.3,60,...,92.0,8,76.4,21,31 : 69,"19,233",38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...


# 3. merge stuffs

In [10]:
# 2 dataframe to 1
pass





# 4. Do correlation?

In [11]:
#work
pass





# 5. Best university? (not EPFL)

In [12]:
#harward or stuff
pass


